In [ ]:
# File        : bench_qwen3_8b_e3_6.ipynb
# Base Model  : Qwen/Qwen3-8B
# EAGLE Model : AngelSlim/Qwen3-8B_eagle3
# EAGLE Type  : EAGLE-3 (use_eagle3=True)
# Quantization: 8-bit (BitsAndBytes)
# Target GPU  : T4 (15GB VRAM)
# Est. VRAM   :
import os
import time
import torch
import gc
import subprocess
import sys
import pandas as pd
from google.colab import userdata

def clear_vram():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    time.sleep(2)

print("=" * 70)
print("🚀 EAGLE BENCHMARK - Qwen3 8B | EAGLE-3 | 8-bit | #6")
print("=" * 70)

try:
    os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
    from huggingface_hub import login
    login(token=os.environ["HF_TOKEN"], add_to_git_credential=False)
except: pass

print("\n📦 Setting up environment...")
subprocess.run("pip uninstall -y transformers bitsandbytes accelerate -q", shell=True, check=False)
subprocess.run("pip install transformers==4.53.1 bitsandbytes>=0.44.0 accelerate==0.26.0 -q", shell=True, check=False)

import shutil
if os.path.exists("EAGLE"):
    shutil.rmtree("EAGLE")
subprocess.run("git clone -q https://github.com/SafeAILab/EAGLE.git", shell=True, check=False)
sys.path.insert(0, os.path.abspath("EAGLE"))

# Patch cnets
cnets = "EAGLE/eagle/model/cnets.py"
if os.path.exists(cnets):
    with open(cnets, 'r') as f: c = f.read()
    if "getattr(config, 'draft_vocab_size'" not in c:
        c = c.replace("self.lm_head=nn.Linear(config.hidden_size,config.draft_vocab_size,bias=False)", "draft_vocab_size = getattr(config, 'draft_vocab_size', config.vocab_size)\n        self.lm_head=nn.Linear(config.hidden_size,draft_vocab_size,bias=False)")
        with open(cnets, 'w') as f: f.write(c)

# NOTE: No Qwen3 stub patch here - we need real Qwen3 support for this benchmark

from transformers import BitsAndBytesConfig, AutoModelForCausalLM
from eagle.model.ea_model import EaModel

BASE_ID = "Qwen/Qwen3-8B"
EAGLE_ID = "AngelSlim/Qwen3-8B_eagle3"

PROMPT = "Write a detailed explanation about how machine learning models work, including the training process, inference, and optimization techniques."

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

from transformers import Qwen2Tokenizer
tokenizer = Qwen2Tokenizer.from_pretrained(BASE_ID, trust_remote_code=True)
messages = [{"role": "user", "content": PROMPT}]
chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(chat, return_tensors="pt").to("cuda")
input_len = inputs.input_ids.shape[1]

results = []

# =============================================================================
# TEST 1: BASELINE
# =============================================================================
clear_vram()
print("\n⚙️  Testing [1/2]: Baseline...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)
model.eval()

with torch.no_grad():
    for _ in range(2):
        _ = model.generate(inputs.input_ids, max_new_tokens=20, do_sample=False)
torch.cuda.synchronize()

torch.cuda.synchronize()
t1 = time.time()
with torch.no_grad():
    out = model.generate(inputs.input_ids, max_new_tokens=100, do_sample=False)
torch.cuda.synchronize()
elapsed = time.time() - t1

tokens = out.shape[1] - input_len
tps = tokens / elapsed
results.append({"Method": "Baseline", "TPS": tps, "Time": elapsed, "Tokens": tokens, "Input": input_len})
print(f"   Generated: {tokens} tokens in {elapsed:.2f}s = {tps:.2f} tok/s")

del model
clear_vram()

# =============================================================================
# TEST 2: EAGLE
# =============================================================================
print("\n🦅 Testing [2/2]: EAGLE...")
eagle = EaModel.from_pretrained(
    base_model_path=BASE_ID,
    ea_model_path=EAGLE_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_eagle3=True,
    low_cpu_mem_usage=True
)
eagle.eval()

with torch.no_grad():
    for _ in range(2):
        _ = eagle.eagenerate(inputs.input_ids, max_new_tokens=20)
torch.cuda.synchronize()

torch.cuda.synchronize()
t1 = time.time()
with torch.no_grad():
    out = eagle.eagenerate(inputs.input_ids, max_new_tokens=100, temperature=0.5)
torch.cuda.synchronize()
elapsed = time.time() - t1

tokens = out.shape[1] - input_len
tps = tokens / elapsed
results.append({"Method": "EAGLE", "TPS": tps, "Time": elapsed, "Tokens": tokens, "Input": input_len})
print(f"   Generated: {tokens} tokens in {elapsed:.2f}s = {tps:.2f} tok/s")

del eagle
clear_vram()

# =============================================================================
# RESULTS
# =============================================================================
df = pd.DataFrame(results)
df['Speedup'] = df['TPS'] / df.iloc[0]['TPS']

print("\n" + "="*70)
print("📊 BENCHMARK RESULTS (100 tokens, longer prompt)")
print("="*70)
print(df.to_string(index=False))
print("="*70)

baseline_tps = df.iloc[0]['TPS']
eagle_tps = df.iloc[1]['TPS']
speedup = df.iloc[1]['Speedup']
time_saved = df.iloc[0]['Time'] - df.iloc[1]['Time']

print(f"\n🎯 Results:")
print(f"   Baseline: {baseline_tps:.2f} tok/s")
print(f"   EAGLE:    {eagle_tps:.2f} tok/s")
print(f"   Speedup:  {speedup:.2f}x")
print(f"   Time saved: {time_saved:.2f}s")

if speedup >= 1.3:
    print("\n✅ EXCELLENT: EAGLE provides significant speedup")
elif speedup >= 1.15:
    print("\n✓ GOOD: EAGLE provides moderate speedup")
elif speedup >= 1.05:
    print("\n⚠️ MODEST: EAGLE provides minimal speedup")
else:
    print("\n❌ ISSUE: EAGLE is slower than baseline")
    print("   Possible causes:")
    print("   - Prompt too short (EAGLE needs longer context)")
    print("   - Generation too short (overhead dominates)")
    print("   - Quantization degrading draft quality too much")

print("="*70)


# =============================================================================
# DIVERGENCE ANALYSIS
# Derives Total Variation Distance (TVD) from the observed speedup.
#
# Theory:
#   In speculative decoding, if the draft model accepts tokens with
#   probability alpha, the expected accepted tokens per step follows:
#       tau = (1 - alpha^(gamma+1)) / (1 - alpha)
#   where gamma is the draft length (5 for EAGLE-3).
#   TVD is then: TVD = 1 - alpha
#   A lower TVD means the draft distribution closely matches the target.
# =============================================================================
gamma    = 5    # EAGLE-3 draft length (tokens proposed per step)
overhead = 0.1  # EAGLE head forward pass overhead (~10% of target model)
tau      = speedup * (1 + overhead)  # Estimated avg accepted tokens/step


def estimate_alpha(target_tau, g, tolerance=0.001):
    """Binary search for acceptance rate alpha given observed tau."""
    low, high = 0.0, 1.0
    for _ in range(20):
        mid = (low + high) / 2
        current_tau = (1 - mid**(g + 1)) / (1 - mid) if mid < 1.0 else g + 1
        if abs(current_tau - target_tau) < tolerance:
            return mid
        if current_tau < target_tau:
            low = mid
        else:
            high = mid
    return low


alpha = estimate_alpha(tau, gamma)
tvd   = 1.0 - alpha

print("\n📐 Divergence Analysis:")
print(f"   Avg Tokens Accepted/Step (τ) : {tau:.2f}")
print(f"   Token Acceptance Rate (α)    : {alpha*100:.1f}%")
print(f"   Total Variation Distance (TVD): {tvd:.4f}")

print("\n💡 Interpretation:")
if tvd < 0.2:
    print(f"   EXCELLENT ALIGNMENT (TVD={tvd:.2f}): Draft head closely mirrors")
    print(f"   the target distribution — high acceptance drives the speedup.")
elif tvd < 0.4:
    print(f"   MODERATE ALIGNMENT (TVD={tvd:.2f}): Draft accepts most tokens")
    print(f"   but diverges on less predictable outputs.")
else:
    print(f"   DIVERGENT (TVD={tvd:.2f}): Draft and target disagree frequently.")
    print(f"   Consider a longer prompt or more generation steps.")

print("="*70)